<a href="https://colab.research.google.com/github/dhanushnayak/Webtrafficcustomlstm/blob/main/WebTrafficCustomLstm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize']=(15,4)
from wordcloud import WordCloud
#del webmodel
!unzip  /content/train_2.csv.zip -d /content

In [ ]:
!pip install livelossplot

In [ ]:
train = pd.read_csv('/content/train_2.csv')

In [ ]:
wordcloud = WordCloud().generate(' '.join(train['Page'].unique().tolist()))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.margins(x=0, y=0)
plt.show()

In [ ]:
start_date =  train.columns[1]
end_date = train.columns[-1]

print(f'Start Date -> {start_date}, End Date -> {end_date}')

In [ ]:
df = train.copy()

In [ ]:
df = df.T

In [ ]:
df.head()

In [ ]:

df.columns = df.iloc[0]
df = df.iloc[2:]

In [ ]:
df.index= pd.to_datetime(df.index)

In [ ]:
from sklearn.preprocessing import StandardScaler,PowerTransformer,MinMaxScaler,FunctionTransformer

std_sc =  FunctionTransformer(np.log1p)
df_s  =  std_sc.transform(df.values.astype(np.float64))


In [ ]:
import os
import pickle
if not os.path.exists('/content/assets/'): os.makedirs('/content/assets')
with open("/content/assets/scaler.pkl",'wb') as f: pickle.dump(std_sc,f)


In [ ]:
df_sr =  pd.DataFrame(df_s,index= pd.to_datetime(df.index)).fillna(0)

In [ ]:
df_sr.mean(axis=0).to_csv('/content/assets/mean_off_sample.csv')

In [ ]:
df_sr-=df_sr.mean(axis=0)


In [ ]:
df_sr = df_sr.iloc[:,:30000]

In [ ]:
train_df = df_sr.iloc[:-1].T
test_df = df_sr.iloc[-60:].T
train_df.shape,test_df.shape

In [ ]:
#del train

In [ ]:
plt.plot(train_df.iloc[0])
plt.plot(test_df.iloc[0])

In [ ]:
test_df.head(5)

In [ ]:
def data_generator(df):
    i=0
    j=60
    while True:
        if i>=len(df)-j-2 or j>=len(df)-2: i,j=0,60
        print(f'I = {i} J = {j}')
        try:
            x= df.iloc[:,i:j].values
            y = df.iloc[:,j].values
            x = x.reshape(x.shape[0],x.shape[1],1)
        except:
          i,j=0,60
          x= df.iloc[:,i:j].values
          y = df.iloc[:,j].values
          x = x.reshape(x.shape[0],x.shape[1],1)
        yield x,y
        i+=1
        j+=1
  

In [ ]:
#x,y = next(data_generator(train_df))

In [ ]:
#x[:,:60,:].shape

In [ ]:
from keras import  Model
from keras.layers import LSTM,Input,Dense,Dropout,Conv1D,Multiply,Add,Layer,Activation,Lambda
import tensorflow as tf





class WebTraffic(Model):
    def __init__(self):
        super(WebTraffic,self).__init__()
        self.inp_data = Input(shape=(None, 1))
        self.out_come = 1
        self.dropout_ratio =  0.3
        self.rates = [2**i for i in range(1,8)]
        self.features =  32
        self.feature_width = 2
        
        
    def slice_layer(self,x,seq_length):
      return x[:,-seq_length:,:]
        
    def call(self,inp,training=None):
      outer_of_cell=[]
      x =  inp
      for rate in self.rates:
        x = Conv1D(16, 1, padding='same', activation='relu')(x)
        x_f = Conv1D(filters=32,
                    kernel_size=2, 
                    padding='causal',
                    dilation_rate=rate,activation='tanh')(x)

        x_g = Conv1D(filters=32,
                    kernel_size=2, 
                    padding='causal',
                    dilation_rate=rate,activation='sigmoid')(x)

        z = Multiply()([(x_f),(x_g)])
        z = Conv1D(16, 1, padding='same', activation='relu')(z)
        x = Add()([x, z]) 

        outer_of_cell.append(z)
      
      out = Activation('relu')(Add()(outer_of_cell))
      print('Out 1 shape -> ',out.shape)

      out = Conv1D(128, 1, padding='same')(out)
      print('Out 2 shape -> ',out.shape)
      out = Activation('relu')(out)
      out = Dropout(.2)(out)
      out = Conv1D(1, 1, padding='same')(out)
      print('Out 3 shape -> ',out.shape)
      pred_seq_train = Lambda(self.slice_layer, arguments={'seq_length':60})(out)
      return pred_seq_train

    
    def build_model(self):
        mod = Model(self.inp_data,outputs=self.call(self.inp_data))
        return mod


#del webmodel 
try: 
  del webmodel
except: pass
webmodel = WebTraffic().build_model()


In [ ]:
train_df_val = np.expand_dims(train_df.values,axis=2)
test_df_val =  np.expand_dims(test_df.values,axis=2)

In [ ]:
class Celloflstm(Layer):
  def __init__(self):
    super().__init__()
    self.rates = [2**i for i in range(1,8)]
  def slice_layer(self,x,seq_length):
      return x[-seq_length:,: :]
  def call(self,ids):
    outer_of_cell=[]
    x = ids
    for rate in self.rates:
        x = Conv1D(16, 1, padding='same', activation='relu')(ids)
        x_f = Conv1D(filters=32,
                    kernel_size=2, 
                    padding='causal',
                    dilation_rate=rate,activation='tanh')(x)

        x_g = Conv1D(filters=32,
                    kernel_size=2, 
                    padding='causal',
                    dilation_rate=rate,activation='sigmoid')(x)

        z = Multiply()([(x_f),(x_g)])
        z = Conv1D(16, 1, padding='same', activation='relu')(z)
        x = Add()([x, z]) 
      
        outer_of_cell.append(z)

    out = Activation('relu')(Add()(outer_of_cell))
    out = Conv1D(128, 1, padding='same')(out)
    print('Out 2 shape -> ',out.shape)
    out = Activation('relu')(out)
    out = Dropout(.2)(out)
    out = Conv1D(1, 1, padding='same')(out)
    print('Out 3 shape -> ',out.shape)
    pred_seq_train = Lambda(self.slice_layer, arguments={'seq_length':60})(out)
    return pred_seq_train


Celloflstm()(train_df_val[0].reshape(train_df_val[0].shape[0],train_df_val[0].shape[1],1)).shape

In [ ]:
#del webmodel
import tensorflow as tf

initial_learning_rate = 0.001

lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate,
    decay_steps=5,
    decay_rate=0.99,
    staircase=True)


In [ ]:
from livelossplot import PlotLossesKeras
from keras.callbacks import EarlyStopping,ModelCheckpoint
ear=EarlyStopping(
    monitor='val_loss',
    min_delta=1e-4,
    patience=25,
    verbose=0,
    mode='auto',
    baseline=None,
    restore_best_weights=False,
)
mod = ModelCheckpoint('webmodel/',
                monitor = 'val_loss',
                save_best_only = True)

callbacks = [PlotLossesKeras(),ear,mod]
webmodel.compile(optimizer=tf.keras.optimizers.Adam(),loss='mean_absolute_error')

In [ ]:
#del webmodel


In [ ]:
batch_size=128
epochs = 500
plt.rcParams['figure.figsize']=(4,3)
webmodel.fit(train_df_val, test_df_val,
                    batch_size=batch_size,
                    epochs=epochs,
                    validation_split=0.3,
                    callbacks =  callbacks,
             )

In [ ]:
id_s = np.random.randint(0,len(df),1)[0]

sample = train_df_val[id_s:id_s+1,:,:]
smple_test = test_df_val[id_s:id_s+1]
print('Sample ',id_s," Size - ",sample.shape)
pred=webmodel.predict(sample).ravel()
print("Pred -> ",pred.shape)


x = train_df.iloc[id_s,:490].values
y = test_df.iloc[id_s,].values
mean_df =  pd.read_csv('/content/assets/mean_off_sample.csv').iloc[:,1]
mean =  mean_df.iloc[id_s]

plt.rcParams['figure.figsize']=(10,3)

plt.plot(np.concatenate([x,pred]),c='b',ls='--')
plt.plot(np.concatenate([x,y]),c='g')
plt.xlim(300,550)


In [ ]:
!unzip  /content/train_1.csv.zip -d /content